TODOs:
- recombine without blowing up numbers (should fit in 64bit word)
- fixedpoint encoding
- truncation operation (using CRT mod operation)
- dot product
- gradient computation
- SDG loop
- compare performance if native type is float64 instead of int64
- performance on GPU

In [1]:
import numpy as np
import tensorflow as tf
from datetime import datetime

Idea below is to simulate five different players on different devices. Hopefully Tensorflow can take care of (optimising?) networking like this.

In [2]:
SERVER_0 = '/device:CPU:0'
SERVER_1 = '/device:CPU:1'
CRYPTO_PRODUCER = '/device:CPU:2'
INPUT_PROVIDER  = '/device:CPU:3'
OUTPUT_RECEIVER = '/device:CPU:4'

config = tf.ConfigProto(
    log_device_placement=True,
    device_count={"CPU": 5},
    inter_op_parallelism_threads=1,
    intra_op_parallelism_threads=1
)

NATIVE_TYPE = tf.int64
# NATIVE_TYPE = tf.float64 # TODO didn't work out of the box

# CRT

In [3]:
m = [89702869, 78489023, 69973811, 70736797, 79637461]
M = 2775323292128270996149412858586749843569 # == prod(ms)

def decompose(x):
    return tuple( x % mi for mi in m )

# *** NOTE ***
# we can recombine without blowing up the results with these huge numbers,
# just need to write port the code from the other notebook.
# Until done, recombine needs to happen outside Tensorflow.

lambdas = [
    875825745388370376486957843033325692983, 
    2472444909335399274510299476273538963924, 
    394981838173825602426191615931186905822, 
    2769522470404025199908727961712750149119, 
    1813194913083192535116061678809447818860
]

def recombine(x):
    return sum( xi * li for xi, li in zip(x, lambdas) ) % M

In [4]:
# *** NOTE ***
# keeping mod operations in-lined here for simplicity;
# we should do them lazily

DOT_OP = tf.matmul

def crt_add(x, y):
    with tf.name_scope("crt_add"):
        return [ (xi + yi) % mi for xi, yi, mi in zip(x, y, m) ]

def crt_sub(x, y):
    with tf.name_scope("crt_sub"):
        return [ (xi - yi) % mi for xi, yi, mi in zip(x, y, m) ]

def crt_mul(x, y):
    with tf.name_scope("crt_mul"):
        return [ (xi * yi) % mi for xi, yi, mi in zip(x, y, m) ]
    
def crt_dot(x, y):
    with tf.name_scope("crt_dot"):
        return [ DOT_OP(xi, yi) % mi for xi, yi, mi in zip(x, y, m) ]

# SPDZ in Tensorflow

In [5]:
class PrivateVariable:
    
    def __init__(self, share0, share1):
        self.share0 = share0
        self.share1 = share1
        
    def __add__(x, y):
        return add(x, y)
    
    def __sub__(x, y):
        return sub(x, y)
    
    def __mul__(x, y):
        return mul(x, y)

In [6]:
def sample(shape):
    with tf.name_scope("sample"):
        return [ tf.random_uniform(shape, maxval=mi, dtype=NATIVE_TYPE) for mi in m ]

def share(secret):
    with tf.name_scope("share"):
        shape = secret[0].shape
        share0 = sample(shape)
        share1 = crt_sub(secret, share0)
        return share0, share1

def reconstruct(share0, share1):
    with tf.name_scope("reconstruct"):
        return crt_add(share0, share1)

In [7]:
def add(x, y):
    assert isinstance(x, PrivateVariable)
    assert isinstance(y, PrivateVariable)
    
    x0, x1 = x.share0, x.share1
    y0, y1 = y.share0, y.share1
    
    with tf.name_scope("add"):
    
        with tf.device(SERVER_0):
            z0 = crt_add(x0, y0)

        with tf.device(SERVER_1):
            z1 = crt_add(x1, y1)

    return PrivateVariable(z0, z1)

In [8]:
def sub(x, y):
    assert isinstance(x, PrivateVariable)
    assert isinstance(y, PrivateVariable)
    
    x0, x1 = x.share0, x.share1
    y0, y1 = y.share0, y.share1
    
    with tf.name_scope("sub"):
    
        with tf.device(SERVER_0):
            z0 = crt_sub(x0, y0)

        with tf.device(SERVER_1):
            z1 = crt_sub(x1, y1)

    return PrivateVariable(z0, z1)

In [9]:
def mul(x, y):
    assert isinstance(x, PrivateVariable)
    assert isinstance(y, PrivateVariable)
    
    x0, x1 = x.share0, x.share1
    y0, y1 = y.share0, y.share1

    with tf.name_scope("mul"):
    
        with tf.device(CRYPTO_PRODUCER):
            a = sample((10,10))
            b = sample((10,10))
            ab = crt_mul(a, b)

            a0, a1 = share(a)
            b0, b1 = share(b)
            ab0, ab1 = share(ab)

        with tf.device(SERVER_0):
            alpha0 = crt_sub(x0, a0)
            beta0  = crt_sub(y0, b0)

        with tf.device(SERVER_1):
            alpha1 = crt_sub(x1, a1)
            beta1  = crt_sub(y1, b1)

        # exchange of alpha's and beta's

        with tf.device(SERVER_0):
            alpha = reconstruct(alpha0, alpha1)
            beta = reconstruct(beta0, beta1)
            z0 = crt_add(ab0,
                 crt_add(crt_mul(a0, beta),
                 crt_add(crt_mul(b0, alpha),
                         crt_mul(alpha, beta))))

        with tf.device(SERVER_1):
            alpha = reconstruct(alpha0, alpha1)
            beta = reconstruct(beta0, beta1)
            z1 = crt_add(ab1,
                 crt_add(crt_mul(a1, beta),
                         crt_mul(b1, alpha)))
        
    return PrivateVariable(z0, z1)

In [10]:
def dot(x, y):
    assert isinstance(x, PrivateVariable)
    assert isinstance(y, PrivateVariable)
    
    x0, x1 = x.share0, x.share1
    y0, y1 = y.share0, y.share1

    with tf.name_scope("dot"):
    
        with tf.device(CRYPTO_PRODUCER):
            a = sample((10,10))
            b = sample((10,10))
            ab = crt_dot(a, b)

            a0, a1 = share(a)
            b0, b1 = share(b)
            ab0, ab1 = share(ab)

        with tf.device(SERVER_0):
            alpha0 = crt_sub(x0, a0)
            beta0  = crt_sub(y0, b0)

        with tf.device(SERVER_1):
            alpha1 = crt_sub(x1, a1)
            beta1  = crt_sub(y1, b1)

        # exchange of alpha's and beta's

        with tf.device(SERVER_0):
            alpha = reconstruct(alpha0, alpha1)
            beta = reconstruct(beta0, beta1)
            z0 = crt_add(ab0,
                 crt_add(crt_dot(a0, beta),
                 crt_add(crt_dot(b0, alpha),
                         crt_dot(alpha, beta))))

        with tf.device(SERVER_1):
            alpha = reconstruct(alpha0, alpha1)
            beta = reconstruct(beta0, beta1)
            z1 = crt_add(ab1,
                 crt_add(crt_dot(a1, beta),
                         crt_dot(b1, alpha)))
        
    return PrivateVariable(z0, z1)

In [11]:
def define_private(shape):
    
    with tf.name_scope("input"):
        
        with tf.device(INPUT_PROVIDER):
            input_x = [ tf.placeholder(NATIVE_TYPE, shape=shape) for _ in range(len(m)) ]
            x = share(input_x)
        
    return input_x, PrivateVariable(*x)

In [12]:
def variable(shape):
    
    with tf.device(SERVER_0):
        x0 = [ tf.Variable(tf.ones(shape=shape, dtype=tf.int64)) for _ in range(len(m)) ]

    with tf.device(SERVER_1):
        x1 = [ tf.Variable(tf.ones(shape=shape, dtype=tf.int64)) for _ in range(len(m)) ]
        
    return PrivateVariable(x0, x1)

In [13]:
# def assign(x):
    
#     x0, x1 = x
    
    
    

In [14]:
def reveal(x):
    assert isinstance(x, PrivateVariable)
    
    x0, x1 = x.share0, x.share1

    with tf.name_scope("reveal"):
    
        with tf.device(OUTPUT_RECEIVER):
            y = reconstruct(x0, x1)
    
    return y

# Simple addition of large numbers

In [15]:
# Inputs
input_x, x = define_private((10,10))
input_w, w = define_private((10,10))
input_b, b = define_private((10,10))

# Computation
z = reveal(mul(x, w) + b)

In [16]:
# Actual inputs
X = np.array([2**100 for _ in range(10*10)]).reshape((10,10))
W = np.array([     2 for _ in range(10*10)]).reshape((10,10))
B = np.array([     0 for _ in range(10*10)]).reshape((10,10))

# Decomposed values outside Tensorflow
inputs = dict(
    [ (xi, Xi) for xi, Xi in zip(input_x, decompose(X)) ] +
    [ (wi, Wi) for wi, Wi in zip(input_w, decompose(W)) ] +
    [ (bi, Bi) for bi, Bi in zip(input_b, decompose(B)) ]
)

run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()

# Run computation using Tensorflow
with tf.Session(config=config) as sess:
    with tf.summary.FileWriter("/tmp/tensorflow/sec", sess.graph) as writer:
        sess.run(tf.global_variables_initializer())
        
        res = sess.run(
            z, 
            inputs,
            options=run_options,
            run_metadata=run_metadata
        )
        
        writer.add_run_metadata(run_metadata, '')

# Recombine result outside Tensorflow
Z = recombine(res)

assert (Z == X * W + B).all(), Z

# Linear regression

In [17]:
    
#     res = sess.run([p], {
#         input_x: np.arange(10*10).reshape(10,10).astype(int),
#     })
# input_x, x = define((10,10))
# 
# w = variable((10,10))
# b = variable((10,10))
# 
# y_pred = add(mul(x, w), b)
# 
# # error = sub(y_pred, y)
# # gradients = 2/m * tf.matmul(tf.transpose(X), error)
# # training_op = tf.assign(theta, theta - learning_rate * gradients)
# 
# p = reveal(y_pred)

#     print(res)

# with tf.Session(config=config) as sess:
#     sess.run(tf.global_variables_initializer())
#     res = sess.run(z, values)
    
#     res = sess.run([p], {
#         input_x: np.arange(10*10).reshape(10,10).astype(int),
#     })
#     print(res)

# Dump